In [155]:
import os
import csv
import itertools

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorlayer as tl

from sklearn.model_selection import train_test_split

from konlpy.tag import Twitter
twitter = Twitter()

In [156]:
data_path = './dataset/chat.in'

In [157]:
#load dataset

delimiter = "\t"

def read_in_data(data_path):
    user = []
    bot = []
    with open(data_path, mode="rt", encoding="utf-8") as fh:
        utt = fh.readlines()
        
        for i, line in enumerate(utt):
            split_line = line.split(delimiter)
            query = split_line[0].replace("\"", "").replace("\n", "")
            answer = split_line[1].replace("\"", "").replace("\n", "")
            
            user.append(query)
            bot.append(answer)
            
        return user, bot
    
user_query, answer_query = read_in_data(data_path)

In [160]:
#데이터 dataset으로 변환 및 문장 쪼개기
dict_data = {'user': user_query, 'bot': answer_query}
df_data = pd.DataFrame(dict_data, columns=['user', 'bot'])

In [162]:
twit_tokenizer = lambda x: twitter.morphs(x)
df_data['user_token'] = df_data['user'].apply(twit_tokenizer) #Tokenizer로 저장
df_data['bot_token'] = df_data['bot'].apply(twit_tokenizer) #Tokenizer로 저장

In [163]:
df_data

,user,bot,user_token,bot_token
0,메롱,저랑 놀고 싶으신거군요,[메롱],"[저, 랑, 놀고, 싶으, 신거, 군요]"
1,아 그러면 안되,네? 어떤 말씀이신지 궁금하네요,"[아, 그러면, 안되]","[네, ?, 어떤, 말씀, 이, 신지, 궁금하네, 요]"
2,오늘 하루 어때?,당신과 함께해서 좋은거 같아요,"[오늘, 하루, 어때, ?]","[당신, 과, 함께해서, 좋, 은, 거, 같아, 요]"
3,너 지금 얼마 있어?,저는 마음만큼은 부자에요.,"[너, 지금, 얼마, 있어, ?]","[저, 는, 마음, 만큼은, 부자, 에요, .]"


In [164]:
#tokenize 된 단어를 합쳐서 사전을 만들어보자
token_dict = itertools.chain.from_iterable(list(df_data['user_token'] + list(df_data['bot_token'])))
token_dict = list(token_dict)

In [217]:
#http://tensorlayer.readthedocs.io/en/latest/
#딕셔너리 만들기

vocab_size = 38

data, count, dictionary, reverse_dictionary = \
        tl.nlp.build_words_dataset(token_dict, vocab_size, True)
    
ids = lambda x: tl.nlp.words_to_word_ids(x, dictionary) #to ids
context = lambda x: tl.nlp.word_ids_to_words(x, reverse_dictionary) #to ids
np_array = lambda x: np.array(x)

Real vocabulary size    38
Limited vocabulary size 38


In [218]:
#token - ids 생성
df_data['user_ids'] = df_data['user_token'].apply(ids).apply(np_array)
df_data['user_ids'].apply(context)

df_data['bot_ids'] = df_data['bot_token'].apply(ids).apply(np_array)
df_data['bot_ids'].apply(context)

0            [저, 랑, 놀고, 싶으, 신거, 군요]
1    [네, ?, 어떤, 말씀, 이, 신지, 궁금하네, 요]
2     [당신, 과, 함께해서, 좋, 은, 거, 같아, 요]
3      [저, 는, 마음, 만큼은, 부자, 에요, UNK]
Name: bot_ids, dtype: object

In [174]:
#Vocab Dictionary 생성하기, most frequent word
tl.nlp.create_vocabulary('./dataset/test5.in', './dataset/chat.in', max_vocabulary_size=10)

Creating vocabulary ./dataset/test5.in from data ./dataset/chat.in


In [177]:
tl.prepro.sequences_add_end_id(list(df_data['user_ids']), end_id='EOS') # 단순히 add end_ID

[[4, 'EOS'],
 [10, 11, 12, 'EOS'],
 [19, 20, 21, 1, 'EOS'],
 [29, 30, 31, 32, 1, 'EOS']]

In [154]:
tl.prepro.sequences_add_end_id(list(df_data['user_ids']), end_id=999) # 단순히 add end_ID
tl.prepro.sequences_add_start_id(list(df_data['user_ids']), start_id=123) # 단순히 add end_ID
tl.prepro.sequences_add_end_id_after_pad(list(df_data['user_ids']), end_id=99, pad_id=0) #add end_ID + padding, 근데 잘 안됨
tl.prepro.sequences_get_mask(list(df_data['user_ids']), pad_val=0) #이것도 왜..?ㄴ

array([1, 1, 1, 1], dtype=object)

In [ ]:
    
    
    tf.estimator.inputs.numpy_input_fn(
    x={"query": np.array(df_data['user_ids'])},
    y=np.array(df_data['bot_ids']),
    num_epochs=None,
    shuffle=True)

In [242]:
np.array(df_data['user_ids'])

array([array([4]), array([10, 11, 12]), array([19, 20, 21,  1]),
       array([29, 30, 31, 32,  1])], dtype=object)

In [239]:
def train_input_fn():
    
    user = tf.constant(np.array(df_data['user_ids']), tf.float32)
    bot = tf.constant(np.array(df_data['bot_ids']), tf.float32)
    
    return user, bot

In [240]:
train_input_fn()

ValueError: setting an array element with a sequence.

In [234]:
def model(features, labels, mode):
    layer1 = tf.layers.dense(features, 10)
    layer2 = tf.layers.dense(layer1, 10)
    layer3 = tf.layers.dense(layer2, 10)
    layer4 = tf.layers.dense(layer3, 10)
    out = tf.layers.dense(layer4, 1)
    
    global_step = tf.train.get_global_step()
    loss = tf.losses.sigmoid_cross_entropy(labels, out)
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss, global_step)
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

In [235]:
est = tf.estimator.Estimator(model)
est.train(train_input_fn)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmpzjx7elb8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13b3ef438>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/ty/43yv2fzx4l9df6zcch8zml6m0000gn/T/tmpzjx7elb8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x13b3ef438>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


AttributeError: 'dict' object has no attribute 'dtype'

In [231]:
train_input_fn()

({'query': <tf.Tensor 'random_shuffle_queue_DequeueMany:1' shape=(128,) dtype=string>},
 <tf.Tensor 'random_shuffle_queue_DequeueMany:2' shape=(128,) dtype=string>)

In [ ]:
model = tf.estimator.DNNClassifier()

# tf.record로 저장하는 것은 binary 파일이며, 이미지나 각 문서를 열때 걸리는 시간을 최소화

In [184]:
def _int64_feature(value):
    return tf.train.Feature(bytes_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [195]:
df_data['user_ids']

0                    [4]
1           [10, 11, 12]
2        [19, 20, 21, 1]
3    [29, 30, 31, 32, 1]
Name: user_ids, dtype: object

In [180]:
train_tfrecord_dir = './dataset/corpus_set.tfrecord'

dataset_writer = tf.python_io.TFRecordWriter(train_tfrecord_dir)

In [ ]:
feature_names = ['user']

In [191]:
features = {'train/query': _bytes_feature(tf.compat.as_bytes(list(df_data['user_ids']))),
           'train/bot': _bytes_feature(tf.compat.as_bytes(list(df_data['bot_ids'])))}

TypeError: Expected binary or unicode string, got [[4], [10, 11, 12], [19, 20, 21, 1], [29, 30, 31, 32, 1]]

In [192]:
embed_size = 10
window_size = 3
dict_size = 400
batch_size = 32
max_length = 30
filter_size = window_size*embed_size

summ = 50

In [ ]:
def model_fn(mode, features, labels):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    embeddings = tf.Variable(
            tf.random_uniform([dict_size, embed_size], seed=7777),
            trainable=False)
    flags = tf.stack([
        tf.zeros(embed_size),
        0.1*tf.ones(embed_size),
        0.2*tf.ones(embed_size)])

    embeddings = tf.assign(embeddings[:3], flags)

    # embedding
    input_embeds = tf.nn.embedding_lookup(embeddings, features['ids'])

    input_flat = tf.layers.flatten(input_embeds)
    input_flat = tf.expand_dims(input_flat, -1)

    init_word = tf.nn.embedding_lookup(embeddings, features['init_word'])
    init_word = tf.layers.flatten(init_word)
    init_word = tf.expand_dims(init_word, -1)

    if not PREDICT:
        label_onehot = tf.one_hot(labels, depth=dict_size, dtype=tf.float32)

    # encoder
    encoder_conv1 = tf.layers.conv1d(
            inputs=input_flat,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size,
            padding='same')

    encoder_conv2 = tf.layers.conv1d(
            inputs=encoder_conv1,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size,
            padding='same')

    encoder_conv3 = tf.layers.conv1d(
            inputs=encoder_conv2,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size,
            padding='same')

    encoder_conv4 = tf.layers.conv1d(
            inputs=encoder_conv3,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size,
            padding='same')

    encoder_glu = encoder_conv4[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv4[:, :, :embed_size])

    #decoder
    next_ids = []
    outs = []
    for l in range(max_length):
        if l == 0:
            decoder_input = init_word
            reuse = False
        else:
            reuse = True

        decoder_conv1 = tf.layers.conv1d(
                inputs=decoder_input,
                filters=2*embed_size,
                kernel_size=filter_size,
                strides=embed_size,
                reuse=reuse,
                name='decoder_1')

        print('decoder conv1 shape ', decoder_conv1.shape)

        decoder_conv1 = decoder_conv1 + decoder_input

        print('decoder conv1 res shape ', decoder_conv1.shape)

        decoder_conv2 = tf.layers.conv1d(
                inputs=decoder_conv1,
                filters=2*embed_size,
                kernel_size=filter_size,
                strides=embed_size,
                reuse=reuse,
                name='decoder_2')

        print('decoder conv2 shape ', decoder_conv2.shape)

        decoder_conv2 = decoder_conv2 + decoder_conv1

        print('decoder conv2 res shape ', decoder_conv2.shape)

        decoder_conv3 = tf.layers.conv1d(
                inputs=decoder_conv2,
                filters=2*embed_size,
                kernel_size=filter_size,
                strides=embed_size,
                reuse=reuse,
                name='decoder_3')

        print('decoder conv3 shape ', decoder_conv3.shape)

        decoder_conv3 = decoder_conv3 + decoder_conv2

        print('decoder conv3 shape ', decoder_conv3.shape)

        decoder_conv4 = tf.layers.conv1d(
                inputs=decoder_conv3,
                filters=2*embed_size,
                kernel_size=filter_size,
                strides=embed_size,
                reuse=reuse,
                name='decoder_4')

        print('decoder conv4 shape ', decoder_conv4.shape)


        decoder_glu = decoder_conv4[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv4[:, :, :embed_size])

        tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1])

        dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)

        attention = tf.nn.softmax(dot_prod, dim=1)

        z_plus_e = encoder_glu + input_embeds

        tiled_attention = tf.tile(attention, [1, 1, embed_size])

        c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
        decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

        logits = tf.layers.dense(c+decoder_glu, dict_size)

        out = tf.nn.softmax(logits)

        next_id = tf.argmax(out, axis=1)
        next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
        next_embeds = tf.expand_dims(next_embeds, -1)
        decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1)

        next_ids.append(next_id)
        outs.append(logits)

    outs = tf.stack(outs, axis=1)
    next_ids = tf.stack(next_ids, axis=1)
    answer_length = tf.reduce_mean(max_length-features['answer_length'], axis=-1)
    sequence_mask = tf.sequence_mask(answer_length, max_length)
    sequence_mask = tf.logical_not(sequence_mask)
    sequence_mask = tf.cast(sequence_mask, tf.float32)

#
#    sequence_mask = tf.Print(sequence_mask,
#            [out, next_id], summarize=100)

    if TRAIN:
        global_step = tf.train.get_global_step()
        losses = tf.nn.softmax_cross_entropy_with_logits(labels=label_onehot, logits=outs)
        loss = tf.reduce_mean(losses * sequence_mask)

        train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                train_op=train_op,
                loss=loss)

    elif EVAL:
        losses = tf.nn.softmax_cross_entropy_with_logits(labels=label_onehot, logits=outs)
        loss = tf.reduce_mean(losses * sequence_mask)

        eval_metric_ops = {'acc': tf.metrics.accuracy(labels, next_ids)}
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=loss,
                eval_metric_ops=eval_metric_ops)

    elif PREDICT:
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={
                    'prediction': next_ids,
                    'contents': features['ids'],
                    'answer': features['label'],
                    'length': features['answer_length']})


    else:
        raise Exception('estiamtor spec is invalid')

    return estimator_spec


def training(est):
    est.train(train_input_fn)


def evaluate(est):
    id_map, word_map = read_maps()
    FLAGS = re.compile('[^PAD|<START>|<END>]')

    pred = est.predict(test_input_fn)
    for p in pred:
        pred_start = -p['length'][0]
        words_cont = map(lambda i: word_map[i], p['contents'])
        words_pred = map(lambda i: word_map[i], p['prediction'][pred_start:])
        words_ans = map(lambda i: word_map[i], p['answer'][pred_start:])

        filtered_contents = ' '.join(filter(FLAGS.match, words_cont))
        filtered_pred = ' '.join(filter(FLAGS.match, words_pred))
        filtered_ans = ' '.join(filter(FLAGS.match, words_ans))

        print('contents: ', filtered_contents)
        print('pred ans: ', filtered_pred)
        print('human ans: ', filtered_ans)
        print()


def main(_):
    config = tf.estimator.RunConfig()
    config._save_checkpoints_steps = 1000
    config._save_checkpoints_secs = None
    fairseq = tf.estimator.Estimator(model_fn, model_dir='./checkpoint/model', config=config)
    for i in range(999999):
        training(fairseq)
        evaluate(fairseq)


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run()

In [101]:
def gen_dataset():
    with open("./dataset/test.in", 'w') as f:
        for i in range(len(df_data['user_ids'])):
            f.write(str(df_data['user_ids'][i]) + '\n')
            f.write(str(df_data['bot_ids'][i]) + '\n')        

gen_dataset()

In [103]:
dataset = tf.data.TextLineDataset("./dataset/test.in")
tr_data = tf.data.Dataset.from_tensor_slices((user_query, answer_query))

iterator = tf.data.Iterator.from_structure(tr_data.output_types,
                                   tr_data.output_shapes)
next_element = iterator.get_next()
training_init_op = iterator.make_initializer(tr_data)

In [99]:
#https://github.com/vahidk/EffectiveTensorflow
dataset = tf.data.TextLineDataset("./dataset/test.in")

tr_data = tf.data.Dataset.from_tensor_slices((user_query, answer_query))

iterator = tf.data.Iterator.from_structure(tr_data.output_types,
                                   tr_data.output_shapes)
next_element = iterator.get_next()
training_init_op = iterator.make_initializer(tr_data)

In [104]:
with tf.Session() as sess:

    # initialize the iterator on the training data
    sess.run(training_init_op)

    # get each element of the training dataset until the end is reached
    while True:
        try:
            elem = sess.run(next_element)
            print(elem)
        except tf.errors.OutOfRangeError:
            print("End of training dataset.")
            break

(b'\xeb\xa9\x94\xeb\xa1\xb1', b'\xec\xa0\x80\xeb\x9e\x91 \xeb\x86\x80\xea\xb3\xa0 \xec\x8b\xb6\xec\x9c\xbc\xec\x8b\xa0\xea\xb1\xb0\xea\xb5\xb0\xec\x9a\x94')
(b'\xec\x95\x84 \xea\xb7\xb8\xeb\x9f\xac\xeb\xa9\xb4 \xec\x95\x88\xeb\x90\x98', b'\xeb\x84\xa4? \xec\x96\xb4\xeb\x96\xa4 \xeb\xa7\x90\xec\x94\x80\xec\x9d\xb4\xec\x8b\xa0\xec\xa7\x80 \xea\xb6\x81\xea\xb8\x88\xed\x95\x98\xeb\x84\xa4\xec\x9a\x94')
(b'\xec\x98\xa4\xeb\x8a\x98 \xed\x95\x98\xeb\xa3\xa8 \xec\x96\xb4\xeb\x95\x8c?', b'\xeb\x8b\xb9\xec\x8b\xa0\xea\xb3\xbc \xed\x95\xa8\xea\xbb\x98\xed\x95\xb4\xec\x84\x9c \xec\xa2\x8b\xec\x9d\x80\xea\xb1\xb0 \xea\xb0\x99\xec\x95\x84\xec\x9a\x94')
(b'\xeb\x84\x88 \xec\xa7\x80\xea\xb8\x88 \xec\x96\xbc\xeb\xa7\x88 \xec\x9e\x88\xec\x96\xb4?', b'\xec\xa0\x80\xeb\x8a\x94 \xeb\xa7\x88\xec\x9d\x8c\xeb\xa7\x8c\xed\x81\xbc\xec\x9d\x80 \xeb\xb6\x80\xec\x9e\x90\xec\x97\x90\xec\x9a\x94.')
End of training dataset.


In [140]:
#Data split
X_train, X_test, y_train, y_test = train_test_split(df_data['user_ids'], df_data['bot_ids'], test_size=0.3,random_state=1)

Real vocabulary size    8
Limited vocabulary size 8
